# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [24]:
# Write your code below.
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the path to the PRICE_DATA folder
price_data_path = os.getenv("PRICE_DATA")
print(f"PRICE_DATA path is: {price_data_path}")

PRICE_DATA path is: ../../05_src/data/prices/


In [23]:
import dask.dataframe as dd

from dotenv import load_dotenv
import os

load_dotenv()
print("PRICE_DATA =", os.getenv("PRICE_DATA"))

PRICE_DATA = ../../05_src/data/prices/


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [28]:
import os
from glob import glob

glob(os.path.join(os.getenv('PRICE_DATA'), "**/*.parquet"), recursive=True)


['../../05_src/data/prices/BHAT/BHAT_2016/part.0.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2016/part.1.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2011/part.0.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2011/part.1.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2018/part.0.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2018/part.1.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2020/part.0.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2020/part.1.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2019/part.0.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2019/part.1.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2010/part.0.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2010/part.1.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2017/part.0.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2017/part.1.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2009/part.0.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2009/part.1.parquet',
 '../../05_src/data/prices/BHAT/BHAT_2007/part.0.parquet

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [27]:
print(ddf.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'Year', 'Close_lag_1'],
      dtype='object')


In [43]:
# Write your code below.

import dask.dataframe as dd
import os
from glob import glob

parquet_paths = glob(os.path.join(os.getenv('PRICE_DATA'), "**/*.parquet"), recursive=True)

ddf = dd.read_parquet(parquet_paths[0:10])

#ddf = ddf.sort_values(by=["ticker", "Date"])


ddf["Close_lag_1"] = ddf.groupby("ticker")["Close"].shift(1)
ddf["Adj_Close_lag_1"] = ddf.groupby("ticker")["Adj Close"].shift(1)

# Step 7: Calculate returns
ddf["returns"] = (ddf["Close"] / ddf["Close_lag_1"]) - 1

# Step 8: Calculate high-low range
ddf["hi_lo_range"] = ddf["High"] - ddf["Low"]

# Step 9: Assign to dd_feat
dd_feat = ddf


/var/folders/r3/cgyvjj3s1890sf6tq0lc4xqh0000gn/T/ipykernel_1893/819148944.py:14: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Close_lag_1"] = ddf.groupby("ticker")["Close"].shift(1)
/var/folders/r3/cgyvjj3s1890sf6tq0lc4xqh0000gn/T/ipykernel_1893/819148944.py:15: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Adj_Close_lag_1"] = ddf.groupby("ticker")["Adj Close"].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [47]:
# Write your code below.

# Convert Dask DataFrame to Pandas
pdf = dd_feat.compute()

# df = pdf.sort_values(by=["ticker", "Date"])
pdf.groupby("ticker", group_keys=False).apply(lambda x: x.assign(Roll_Avg_10 = x['returns'].rolling(10).mean()))

/var/folders/r3/cgyvjj3s1890sf6tq0lc4xqh0000gn/T/ipykernel_1893/478182351.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pdf.groupby("ticker", group_keys=False).apply(lambda x: x.assign(Roll_Avg_10 = x['returns'].rolling(10).mean()))


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,Roll_Avg_10
2123,2020-02-24,1.130000,1.130000,0.990000,1.010000,1.010000,241200,BHAT.csv,BHAT,2020,1.120000,1.120000,-0.098214,0.14,NaN
2142,2020-03-20,0.870000,0.900000,0.760000,0.811000,0.811000,119000,BHAT.csv,BHAT,2020,0.910000,0.910000,-0.108791,0.14,NaN
2145,2020-03-25,0.780000,0.800000,0.750000,0.780000,0.780000,49000,BHAT.csv,BHAT,2020,0.790000,0.790000,-0.012658,0.05,NaN
2149,2020-03-31,0.720000,0.820000,0.720000,0.810000,0.810000,61200,BHAT.csv,BHAT,2020,0.711000,0.711000,0.139240,0.10,NaN
2040,2019-10-23,3.260000,3.450000,2.990000,3.060000,3.060000,183000,BHAT.csv,BHAT,2019,3.360000,3.360000,-0.089286,0.46,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554,2016-05-24,671.000000,671.000000,671.000000,671.000000,671.000000,3000,BHAT.csv,BHAT,2016,700.109985,700.109985,-0.041579,0.00,-0.001461
1564,2016-06-08,700.000000,700.000000,700.000000,700.000000,700.000000,0,BHAT.csv,BHAT,2016,700.000000,700.000000,0.000000,0.00,0.003538
1568,2016-06-14,700.000000,700.000000,700.000000,700.000000,700.000000,0,BHAT.csv,BHAT,2016,700.000000,700.000000,0.000000,0.00,0.008537
1959,2018-01-03,692.840027,692.840027,692.840027,692.840027,692.840027,0,BHAT.csv,BHAT,2018,692.840027,692.840027,0.000000,0.00,0.008537


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Not necessary, but converting to pandas can make the task easier depending on the complexity and the size of your dataset.
+ Would it have been better to do it in Dask? Why?
	•	The syntax is more complex than in pandas.
	•	It requires setting the index properly (usually with set_index()), which can be expensive.
	•	Some Dask versions have limited support for rolling operations with groupby.
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.